In [1]:
pip install geopy 

Note: you may need to restart the kernel to use updated packages.


In [2]:
# 1
# removing duplciate lane entries
# removing lane info "_lane" from dataset
import pandas as pd

def clean_dataset(df):
    # Group the data by the two columns and keep only the first occurrence of each group
    reduced_df = df.drop_duplicates(subset=[5, 6], keep='first')
    # Convert column 1 values to strings and remove anything after and including the last underscore
    #reduced_df[1] = reduced_df[1].astype(str).str.rsplit('_', n=1).str[0]
    reduced_df.loc[:, 1] = reduced_df.loc[:, 1].astype(str).str.rsplit('_', n=1).str[0]

    reduced_df.iloc[:, 5] = reduced_df.iloc[:, 5].astype(str).apply(lambda x: float(x[:2] + '.' + x[2:]))
    reduced_df.iloc[:, 6] = reduced_df.iloc[:, 6].astype(str).apply(lambda x: float(x[:4] + '.' + x[4:]))

    return reduced_df


In [3]:
# 2
# group data based on sensor location description 
import pandas as pd

def get_sensor_group(data, sensor_loc, ignore=None):
    # Filter the DataFrame based on column 3 and ignore string
    mask = data[3].str.startswith(sensor_loc) & (~data[3].str.contains(ignore) if ignore is not None else True)
    matching_group = data[mask]
    return matching_group


In [4]:
import folium
from geopy.distance import distance
import pandas as pd

def plot_min_dis(data):
    latitude = data.iloc[:, 5]
    longitude = data.iloc[:, 6]
    description = data.iloc[:, 3]
    sensor_data = pd.DataFrame({'Latitude': latitude, 'Longitude': longitude, 'Description': description})

    map = folium.Map(location=[latitude.mean(), longitude.mean()], zoom_start=10)

    # Iterate over the rows and add a circle marker for each location
    for index, row in sensor_data.iterrows():
        lat = row['Latitude']
        lon = row['Longitude']
        description = row['Description']
        tooltip_text = f"Latitude: {lat}, Longitude: {lon}, Description: {description}"
        folium.Circle(location=[lat, lon], radius=40, color='blue', fill=True, fill_color='blue',
                      tooltip=tooltip_text, sticky=True).add_to(map)

    # Connect nodes with the shortest distance
    for i in range(len(sensor_data)):
        current_node = sensor_data.iloc[i]
        current_lat = current_node['Latitude']
        current_lon = current_node['Longitude']

        min_distance = float('inf')
        min_index = -1

        for j in range(len(sensor_data)):
            if i != j:
                target_node = sensor_data.iloc[j]
                target_lat = target_node['Latitude']
                target_lon = target_node['Longitude']

                # Calculate distance using Vincenty formula
                dist = distance((current_lat, current_lon), (target_lat, target_lon)).meters

                if dist < min_distance:
                    min_distance = dist
                    min_index = j

        if min_index != -1:
            target_node = sensor_data.iloc[min_index]
            target_lat = target_node['Latitude']
            target_lon = target_node['Longitude']

            # Connect nodes with a polyline
            folium.PolyLine(locations=[(current_lat, current_lon), (target_lat, target_lon)], color='red').add_to(map)

    return map

In [5]:
import pandas as pd
import folium
from geopy.distance import distance

def calculate_distance(sensor_data):
    # Calculate distances between each pair of sensors
    num_sensors = len(sensor_data)
    distances = [[0.0] * num_sensors for _ in range(num_sensors)]
    for i in range(num_sensors):
        for j in range(i+1, num_sensors):
            coord1 = (sensor_data.iloc[i][5], sensor_data.iloc[i][6])
            coord2 = (sensor_data.iloc[j][5], sensor_data.iloc[j][6])
            dist = distance(coord1, coord2).meters
            distances[i][j] = dist
            distances[j][i] = dist
    return distances

def tsp(sensor_data):
    
    distances = calculate_distance(sensor_data)
    map = folium.Map(location=[sensor_data[5].mean(), sensor_data[6].mean()], zoom_start=10)

    # Connect sensors with the shortest distance
    num_sensors = len(sensor_data)
    connected = set()
    min_distance = float('inf')
    min_i, min_j = -1, -1

    # Find the pair of sensors with the shortest distance
    for i in range(num_sensors):
        for j in range(i+1, num_sensors):
            if distances[i][j] < min_distance:
                min_distance = distances[i][j]
                min_i, min_j = i, j

    # Connect the pair of sensors with a polyline
    if min_i != -1 and min_j != -1:
        sensor1 = sensor_data.iloc[min_i]
        sensor2 = sensor_data.iloc[min_j]
        folium.PolyLine([(sensor1[5], sensor1[6]), (sensor2[5], sensor2[6])],
                        color='red').add_to(map)
        connected.add(min_i)
        connected.add(min_j)

    # Connect the remaining sensors
    while len(connected) < num_sensors:
        min_distance = float('inf')
        min_i = -1
        for i in range(num_sensors):
            if i not in connected:
                for j in connected:
                    if distances[i][j] < min_distance:
                        min_distance = distances[i][j]
                        min_i = i
                        min_j = j
        if min_i != -1:
            sensor1 = sensor_data.iloc[min_i]
            sensor2 = sensor_data.iloc[min_j]
            folium.PolyLine([(sensor1[5], sensor1[6]), (sensor2[5], sensor2[6])],
                            color='red').add_to(map)
            connected.add(min_i)

    
    for _, sensor in sensor_data.iterrows():
        folium.CircleMarker(
            location=[sensor[5], sensor[6]],
            radius=5,
            color='blue',
            fill=True,
            fill_color='blue').add_to(map)

    return map



In [8]:
# Run

data = clean_dataset(pd.read_csv('../detector/detectors2018.csv', header=None))

#sensor_loc = "CC-215 WB"
#sensor_loc="CC-215 EB"
#sensor_loc="I-15 NB"
sensor_loc= "I-515 NB"


ignore = '@'
sensor_group = get_sensor_group(data, sensor_loc, ignore)

#sensor_map = plot_dfs(sensor_group) 
#sensor_map = plot_direc(sensor_group)
#sensor_map = plot_tsp(sensor_group)
#sensor_map = plot_min_dis(sensor_group)

sensor_map = tsp(sensor_group)

sensor_map

C:\Users\MSI\AppData\Local\Temp\ipykernel_17036\1195696214.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  reduced_df.loc[:, 1] = reduced_df.loc[:, 1].astype(str).str.rsplit('_', n=1).str[0]
C:\Users\MSI\AppData\Local\Temp\ipykernel_17036\1195696214.py:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  reduced_df.iloc[:, 5] = reduced_df.iloc[:, 5].astype(str).apply(lambda x: float(x[:2] + '.' + x[2:]))
C:\Users\MSI\AppData\Local\Temp\ipykernel_17036\1195696214.py:14: SettingWithCopyWarning: 
A value is 